In [1]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/nasnet.py
import os, re, glob
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import pandas as pd
import tensorflowjs as tfjs

from keras import layers, Model, callbacks
from keras.optimizers import Adam
from keras.applications.nasnet import NASNetMobile
from sklearn.model_selection import train_test_split
from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy
from ohia.encoders import FastLabelEncoder
from ohia.utils import PlantNetGenerator

Using TensorFlow backend.


In [2]:
SEED = 0
BATCH_SIZE = 64
MODEL_NAME = 'nasnetmobile'
FILE_PATH = '/home/matt/repos/ohia.ai/data'
IMAGE_DIR = 'resized_images'

In [3]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]
full_label_ids = FastLabelEncoder().fit_transform(full_label_list)

In [4]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, full_label_ids, test_size=0.1, random_state=SEED)

In [5]:
# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(train_files, train_ids, n_classes, BATCH_SIZE, augment=True)
valid_generator = PlantNetGenerator(valid_files, valid_ids, n_classes, BATCH_SIZE, augment=False)

In [6]:
# load pretrained ImageNet model
base_model = NASNetMobile(
    input_shape=(224,224,3),
    weights='imagenet',
    include_top=False
)

# set freeze all layers
for layer in base_model.layers:
    layer.trainable = False
    
# map ImageNet features to plants
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(n_classes, activation='softmax')(x)
    
# compile the model
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
)

# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{FILE_PATH}/weights/{MODEL_NAME}' + '_{epoch:02d}.hdf5',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
    ),
]

# train model
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=20,
    epochs=100,    
)

Epoch 1/100
165/165 [==============================] - 55s 331ms/step - loss: 2.3791 - top_1_accuracy: 0.3351 - top_3_accuracy: 0.5460 - top_5_accuracy: 0.6527 - val_loss: 2.0403 - val_top_1_accuracy: 0.4227 - val_top_3_accuracy: 0.6727 - val_top_5_accuracy: 0.7830
Epoch 2/100
165/165 [==============================] - 41s 247ms/step - loss: 1.7285 - top_1_accuracy: 0.5083 - top_3_accuracy: 0.7360 - top_5_accuracy: 0.8295 - val_loss: 1.7635 - val_top_1_accuracy: 0.4878 - val_top_3_accuracy: 0.7231 - val_top_5_accuracy: 0.8255
Epoch 3/100
165/165 [==============================] - 41s 247ms/step - loss: 1.4908 - top_1_accuracy: 0.5623 - top_3_accuracy: 0.7888 - top_5_accuracy: 0.8741 - val_loss: 1.6390 - val_top_1_accuracy: 0.5113 - val_top_3_accuracy: 0.7465 - val_top_5_accuracy: 0.8472
Epoch 4/100
165/165 [==============================] - 42s 253ms/step - loss: 1.3586 - top_1_accuracy: 0.5982 - top_3_accuracy: 0.8157 - top_5_accuracy: 0.8937 - val_loss: 1.6083 - val_top_1_accuracy: 0

Process ForkPoolWorker-555:
Process ForkPoolWorker-554:
Process ForkPoolWorker-552:
Process ForkPoolWorker-542:
Process ForkPoolWorker-551:
Process ForkPoolWorker-548:
Process ForkPoolWorker-549:
Process ForkPoolWorker-543:
Process ForkPoolWorker-558:
Process ForkPoolWorker-531:
Process ForkPoolWorker-544:
Process ForkPoolWorker-557:
Process ForkPoolWorker-539:
Process ForkPoolWorker-559:
Process ForkPoolWorker-553:
Process ForkPoolWorker-541:
Process ForkPoolWorker-546:
Process ForkPoolWorker-550:
Process ForkPoolWorker-533:
Process ForkPoolWorker-540:
Process ForkPoolWorker-530:
Process ForkPoolWorker-529:
Process ForkPoolWorker-560:
Process ForkPoolWorker-556:
Process ForkPoolWorker-528:
Process ForkPoolWorker-537:
Process ForkPoolWorker-523:
Process ForkPoolWorker-525:
Process ForkPoolWorker-535:
Process ForkPoolWorker-545:
Process ForkPoolWorker-538:
Process ForkPoolWorker-521:
Process ForkPoolWorker-547:
Process ForkPoolWorker-532:
Process ForkPoolWorker-536:
Process ForkPoolWork

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/sy

  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/hom

  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/matt/anaconda3/envs/kaggle/lib

KeyboardInterrupt: 

In [ ]:
# load best model
best_weights = glob.glob(f'{FILE_PATH}/weights/{MODEL_NAME}**')
best_weights = np.sort(best_weights)[-1]
model.load_weights(best_weights)

In [ ]:
# save model
tfjs.converters.save_keras_model(model, f'{FILE_PATH}/models/{MODEL_NAME}')

In [ ]:
# n_units: 1024,  val_top_1_accuracy: 0.5590,  val_top_3_accuracy: 0.8056,  val_top_5_accuracy: 0.8854